In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

##### Data Cleaning

In [4]:
DATA_PATH = r"data"
STORE = DATA_PATH + '/assets.h5'

In [5]:
def remove_correlated_assets(df, cutoff = 0.99, verbose = False):
    corr = df.corr().stack()
    corr = corr[corr < 1]
    corr_subset = corr[corr.abs() >= cutoff].index
    keep, drop = set(), set()
    for s1, s2 in corr_subset:
        if s1 not in keep:
            if s2 not in keep:
                keep.add(s1)
                drop.add(s2)
            else:
                drop.add(s1)
        else:
            keep.discard(s2)
            drop.add(s2)
    df = df.drop(drop, axis = 1)
    if verbose:
        return [df, (corr, drop)]
    return df
    

In [6]:
def check_stationarity(df):
    results = []
    for ticker, prices in df.items():
        results.append([ticker, adfuller(prices, regression='ct')[1]])
    return pd.DataFrame(results, columns = ['ticker', 'adf']).sort_values('adf')

In [7]:
def remove_stationary_assets(df, pval = 0.05, verbose = False):
    test_result = check_stationarity(df)
    stationary = test_result.loc[test_result.adf <= pval, 'ticker'].tolist()
    if verbose:
        return [df.drop(stationary, axis = 1), stationary]
    return df.drop(stationary, axis = 1)

In [8]:
def select_assets(asset_class='stocks', n = 500, start = 2013, end = 2023):
    idx = pd.IndexSlice
    with pd.HDFStore(STORE) as store:
        df = (pd.concat([store[f'stooq/us/nasdaq/{asset_class}/prices'],
                        store[f'stooq/us/nyse/{asset_class}/prices']])
             .loc[lambda df: ~df.index.duplicated()]
             .sort_index()
             .loc[idx[:, f'{start}':f'{end}'], :]
             .assign(dv = lambda df: df.close.mul(df.volume)))

    most_traded = (df.groupby(level = 'ticker')
                   .dv.mean()
                   .nlargest(n=n).index)

    df = (df.loc[idx[most_traded, :], 'close']
          .unstack('ticker')
          .ffill(limit = 5)
          .dropna(axis = 1))

    df = remove_correlated_assets(df)
    df = remove_stationary_assets(df)
    return df


In [8]:
for asset_class, n in [('etfs', 500), ('stocks', 250)]:
    df = select_assets(asset_class = asset_class, n = n)
    df.to_hdf(DATA_PATH + r'/etfs_stocks_universe.h5', f'{asset_class}')

In [14]:
all_df = pd.DataFrame()
for asset_class, n in [('etfs', 500), ('stocks', 250)]:
    with pd.HDFStore(STORE) as store:
        df = (pd.concat([store[f'stooq/us/nasdaq/{asset_class}/prices'],
                        store[f'stooq/us/nyse/{asset_class}/prices']])
                .loc[lambda df: ~df.index.duplicated()]
                .sort_index())
                # .loc[idx[:, f'{start}':f'{end}'], :]
                # .assign(dv = lambda df: df.close.mul(df.volume)))
    all_df = pd.concat([all_df, df])


Candidate Selection

In [10]:
DATA_PATH = r"data"
STORE = DATA_PATH + r"\backtest.h5"

In [11]:
critical_values = {0: {0.9: 13.4294, 0.95: 15.4943, 0.99: 19.9349},
                   1: {0.9: 2.7055, 0.95: 3.8415, 0.99: 6.6349}}

trace0_cv = critical_values[0][0.99]
trace1_cv = critical_values[1][0.99]

In [13]:
with pd.HDFStore(STORE) as store:
    cointegration_res = store['cointegration_test']

In [17]:
def select_candidate_pairs(df):
    
    df['joh_sig'] = ((df['trace0'] > trace0_cv) & (df['trace1'] > trace1_cv))
    df['eg'] = df[['eg1','eg2']].min(axis = 1) 
    df['s1_dep'] = (df['eg1'] < df['eg2'])
    df['eg_sig'] = df['eg'] < 0.05
    df = df.drop(['eig0', 'eig1', 'w1', 'w2', 'eg1', 'eg2', 'trace0', 'trace1'], axis = 1) 

    candidates = df[df['joh_sig'] | df['eg_sig']]
    candidates['y'] = candidates.apply(lambda x: x['s1'] if x['s1_dep'] else x['s2'], axis = 1)
    candidates['x'] = candidates.apply(lambda x: x['s2'] if x['s1_dep'] else x['s1'], axis = 1)
    candidates = candidates.drop(['s1', 's2', 'joh_sig', 's1_dep', 'eg_sig'], axis = 1)
    
    return candidates

In [18]:
candidates = select_candidate_pairs(cointegration_res)
candidates.to_hdf(STORE, "candidates")

##### Pricing Data

In [ ]:
idx = pd.IndexSlice
DATA_PATH = r"C:\Personal\time-series\data"
STORE = DATA_PATH + '/assets.h5'
with pd.HDFStore(STORE) as store:
    prices = (pd.concat([
        store['stooq/us/nyse/stocks/prices'],
        store['stooq/us/nyse/etfs/prices'],
        store['stooq/us/nasdaq/etfs/prices'],
        store['stooq/us/nasdaq/stocks/prices']])
                .sort_index()
                .loc[idx[:, '2013':'2023'], :])
print(prices.info())
prices.to_hdf('data/backtest.h5', 'prices')

##### Deletion from HDF

In [24]:
# import tables
# with tables.open_file(r'data/backtest.h5', mode = 'r+') as h5_file:
    # h5_file.remove_node('/test1/trades', recursive = True)
    # h5_file.remove_node('/test2/trades', recursive = True)
    # h5_file.remove_node('/test3/trades', recursive = True)
    # h5_file.remove_node('/prices_wide', recursive = True)